![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

The company is specifically interested in how it can improve revenue. Dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

Four datasets to investigate:

###  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

### finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

### info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

### reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

## Explanatory Data Analysis

In [63]:
"""Import necessary modules"""
import pandas as pd

In [64]:
"""Import the datasets"""
info = pd.read_csv("info.csv")
finance = pd.read_csv("finance.csv")
reviews = pd.read_csv("reviews.csv")
brands = pd.read_csv("brands.csv")

In [65]:
"""Merge the data and drop null values"""
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
merged_df.dropna(inplace=True)

In [66]:
"""Add price labels based on listing_price quartiles"""
merged_df["price_label"] = pd.qcut(merged_df["listing_price"], q=4, labels=["Budget", "Average", "Expensive", "Elite"])

In [67]:
"""Group by brand and price_label to get volume and mean revenue"""
adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")).round(2)

print(adidas_vs_nike)

    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45


In [68]:
"""Store the length of each description"""
merged_df["description_length"] = merged_df["description"].str.len()

In [69]:
""" Upper description length limits"""
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]

In [70]:
"""Description length labels"""
labels = ["100", "200", "300", "400", "500", "600", "700"]

In [71]:
"""Cut into bins"""
merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)

In [72]:
# Group by the bins
description_lengths = merged_df.groupby("description_length", as_index=False).agg(
    mean_rating=("rating", "mean"), 
    num_reviews=("reviews", "count")).round(2)

print(description_lengths)

  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15


In [73]:
"""List of footwear keywords"""
mylist = "shoe*|trainer*|foot*"

In [74]:
"""Filter for footwear products"""
shoes = merged_df[merged_df["description"].str.contains(mylist)]

In [75]:
"""Filter for clothing products"""
clothing = merged_df[~merged_df.isin(shoes["product_id"])]

In [76]:
"""Remove null product_id values from clothing DataFrame"""
clothing.dropna(inplace=True)

In [77]:
"""Create product_types DataFrame"""
product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                              "median_clothing_revenue": clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                              "median_footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])

In [78]:
"""Shows the outcome"""
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,478,625.07,2639,3073.3
